# Projet de modélisation d'un mix énergétique issu 100% d'ENR

## III. Simulations du modèle sur l'optimisation du coût du mix énergétique

### Imports

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import csv
import time
import sys

# Imports pour pyomo
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

import panel as pn 
from ipywidgets import interactive, interact_manual
import ipywidgets as widgets

#%matplotlib inline
pn.extension()

from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvas

## 1. Utils

In [2]:
col = {"offshore" : '#1f4e79', "onshore" : '#548235' , "pv" : "#ffc000", "phs" : "#bdd7ee", "battery" : "#7030a0", "biogas" :"#843c0c", "methanation": "#7f6000", "gas": "grey", "lake": "#4169E1", "river" : "#09F1D3"}
tec = ["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]
stor = ["phs","battery","methanation"]

In [3]:
# Dashboard

def show(week,name):
    
    if name == "scen2006": 
        d = scen2006
        Volume = Volume_str_opti 
    else :
        d = pd.read_csv(name+"/"+name+"_hourly_generation.csv", index_col = 0, squeeze = True)
        Volume = pd.read_csv(name+"/"+name+"_Volume.csv", index_col = 0, squeeze = True)
        
    tec_aux = tec
    if not("river" in d.columns): tec_aux = list(set(tec_aux) - set(["river"]))
    if not("lake" in d.columns): tec_aux = list(set(tec_aux) - set(["lake"]))
    
    h  = 168*(week-1) + np.arange(1,169)
    ymin, ymax = -45,130
    h_ratio = np.ones(len(stor)+1)
    h_ratio[0] = 2 # 1er subplot 2 fois plus grand
    fig, axs = plt.subplots(len(stor)+1, figsize = (15,15),gridspec_kw={'height_ratios':h_ratio})
    
    # Stockage 
    sum_s = 0
    for i in stor: 
        sum_s += d["Storage " + i][h]
    for s in stor[::-1]:
        aux = "Storage " + s
        axs[0].fill_between(h, sum_s, color = col[s], label = aux)
        sum_s -= d[aux][h]

    # Production
    sum_g = 0
    for i in tec_aux: 
        sum_g += d[i][h]
    for gene in tec_aux[::-1]: 
        axs[0].fill_between(h,sum_g, color = col[gene], label = gene )
        sum_g -= d[gene][h]
    
    # Demande
    axs[0].plot(h, d["Electricity demand"][h], color = 'r' , label = "Demand",linewidth= 4)

    axs[0].vlines(x=np.linspace(h[0],h[-1],8),ymin = ymin, ymax = ymax,colors="black",linestyle='dotted',linewidth=2)
    axs[0].set_ylabel("Production (GWh)")
    axs[0].set_ylim(ymin,ymax)
    axs[0].legend(loc ="upper right")
    axs[0].set_title("Semaine "+ str(week))
        
    for s in range(0,len(stor)):
        axs[s+1].fill_between(h, d["Stored " + stor[s]][h], color = col[stor[s]])
        axs[s+1].hlines(y=Volume[s], xmin = h[0], xmax = h[-1], label = "limit", color = 'red')
        axs[s+1].set_title("Stored " + str(stor[s]))
        axs[s+1].vlines(x=np.linspace(h[0],h[-1],8),ymin = 0, ymax = Volume[s]+10,colors="black",linestyle='dotted',linewidth=2)
        
        
def affichage(simu):
    print(" --- Q ---")
    for i in simu.model.Q:
        print(i," : ",pyo.value(simu.model.Q[i]))   
    print(" --- S ---")   
    for i in simu.model.S:
        print(i," : ",pyo.value(simu.model.S[i]))
    print(" --- volume ---")
    for i in simu.model.volume:
        print(i," : ",pyo.value(simu.model.volume[i]))
    print(" --- cost --- ")
    print(simu.cost())

In [4]:
# Optimal values found in summary with demand of 2016.txt
Volume_str_opti = pd.Series([180 ,74.14, 12499.09], index = stor)
S_opti = pd.Series([9.3,20.08, 7.66], index = stor)
Q_tec_opti = pd.Series([12.64, 79.73, 121.98, 7.5, 13 ,32.93, 9.3, 20.08, 32.93], index = tec)
Cost_opti = 21.84 

## 2. Simulation du modèle EOLES

### 2.1. Chois de Q, S et VOLUME

In [5]:
phs_bounds = {"Q_lower":5.2,"Q_upper": 9.3, "S_lower":4.2, "S_upper":9.3, "Volume_lower":80.16, "Volume_upper":180}
vre_upper_bounds = {"onshore":120, "offshore": 20, "pv": 218}

In [6]:
V = widgets.Text(value='Volume de stockage (GWh)', disabled=True)
Volume_phs = widgets.FloatSlider(min=phs_bounds["Volume_lower"], max=phs_bounds["Volume_upper"], step=0.01, description='Volume phs',value = Volume_str_opti["phs"])
Volume_methanation = widgets.FloatSlider(min=0, max=Volume_str_opti["methanation"]*5, step=0.01, description='Volume methanation',value = Volume_str_opti["methanation"])
Volume_battery =  widgets.FloatSlider(min=0, max=Volume_str_opti["battery"]*5, step=0.01, description='Volume battery',value = Volume_str_opti["battery"])  

Q = widgets.Text(value='Capcité installée (GW)', disabled=True)
Q_offshore= widgets.FloatSlider(min=0, max=vre_upper_bounds["offshore"], step=0.01, description='Q offshore',value = Q_tec_opti["offshore"])
Q_onshore= widgets.FloatSlider(min=0, max=vre_upper_bounds["onshore"], step=0.01, description='Q onshore',value = Q_tec_opti["onshore"])
Q_pv= widgets.FloatSlider(min=0, max=vre_upper_bounds["pv"], step=0.1, description='Q pv',value = Q_tec_opti["pv"])
Q_river= widgets.FloatSlider(min=0, max=Q_tec_opti["river"], step=0.01, description='Q river',value = Q_tec_opti["river"])
Q_lake= widgets.FloatSlider(min=12.855, max=13, step=0.01, description='Q lake',value = Q_tec_opti["lake"])
Q_biogas= widgets.FloatSlider(min=0, max=Q_tec_opti["biogas"]*5, step=0.01, description='Q biogas',value = Q_tec_opti["biogas"])
Q_phs= widgets.FloatSlider(min=phs_bounds["Q_lower"], max=phs_bounds["Q_upper"], step=0.01, description='Q phs',value = Q_tec_opti["phs"])
Q_battery= widgets.FloatSlider(min=0, max=Q_tec_opti["battery"]*5, step=0.01, description='Q battery',value = Q_tec_opti["battery"])
Q_methanation= widgets.FloatSlider(min=0, max=Q_tec_opti["methanation"]*5, step=0.01, description='Q methanation',value = Q_tec_opti["methanation"])

S = widgets.Text(value='Capacité de charge (GW)', disabled=True)
S_phs= widgets.FloatSlider(min=phs_bounds["S_lower"], max=phs_bounds["S_upper"], step=0.01, description='S phs',value = S_opti["phs"])
S_methanation = widgets.FloatSlider(min=0, max=S_opti["methanation"]*5, step=0.01, description='S methanation',value = S_opti["methanation"])
warn = widgets.Textarea(value='      Attention ! \n      Volume > Q  et  Volume > S', disabled=True)

widgets.HBox([widgets.VBox([Q, Q_offshore, Q_onshore, Q_pv, Q_river, Q_lake,Q_biogas ,Q_phs,Q_battery,Q_methanation]),
widgets.VBox([V,Volume_phs,Volume_battery,Volume_methanation, warn]), widgets.VBox([S,S_methanation, S_phs])])


In [7]:
#Valeurs modifiées par les curseurs :
Q_tec = [Q_offshore.value, Q_onshore.value, Q_pv.value, Q_river.value, Q_lake.value,Q_biogas.value ,Q_phs.value,Q_battery.value,Q_methanation.value]
Volume_str = [Volume_phs.value,Volume_battery.value,Volume_methanation.value]
S =  [Q_phs.value,Q_battery.value,S_methanation.value]


# Capacity in GW Q
Q_tec = pd.Series( Q_tec, index = tec)

    
# Energy volume of storage technology in GWh VOLUME
Volume_str = pd.Series(Volume_str, index = stor)

#Storage charging power S
S = pd.Series(S, index = stor)


print("Fixed values : ")
print("--------- Q ---------")
print(Q_tec)
print("-------- VOLUME -------")
print(Volume_str)
print("--------- S ---------")
print(S)

Fixed values : 
--------- Q ---------
offshore        12.64
onshore         79.73
pv             121.98
river            7.50
lake            13.00
biogas          32.93
phs              9.30
battery         20.08
methanation     32.93
dtype: float64
-------- VOLUME -------
phs              180.00
battery           74.14
methanation    12499.09
dtype: float64
--------- S ---------
phs             9.30
battery        20.08
methanation     7.66
dtype: float64


### 2.2 Simulation avec les paramètres optimaux de l'année 2006

In [8]:
from modeles import modele

### <span style="color:red">Attention</span>
Il faut modifier le fichier modele.py en choisissant le scénario de demande que vous souhaitez simuler.  
Penez à décommenter le commentaire associé à la demande pour vérifier que le fichier importé a bien été modifié.  
Il faut relancer le noyau à chaque nouvel expérience sinon jupyter ne prend pas les dernières modifications.

#### Demande de RTE  ERREUR ici j'ai affiché les données de negawatt je suis en train de travailler sur ce pb

In [9]:
#Run the simulation
simu0 = modele(Q_tec, Volume_str, S)
simu0.load_param("inputs/")
simu0.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
Demande de Negawatt
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
Simulation du modèle faite avec succès ! 


In [10]:
#Write results
res0 = simu0.write_results('simu_opti_rte')

Ecriture des résultats ...


#### Demande de l'ADEME

In [9]:
#Run the simulation
simu1 = modele(Q_tec, Volume_str, S)
simu1.load_param("inputs/")
simu1.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
Demande de l'ADEME
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
Simulation du modèle faite avec succès ! 


In [10]:
#Write results
res1 = simu1.write_results('simu_opti_ademe')

Ecriture des résultats ...


#### Demande de Négawatt

In [9]:
#Run the simulation
simu2 = modele(Q_tec, Volume_str, S)
simu2.load_param("inputs/")
simu2.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
Demande de Negawatt
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
Simulation du modèle faite avec succès ! 


In [11]:
#Write results
res2 = simu2.write_results('simu_opti_negawatt')

Ecriture des résultats ...


### 2.3 Analyse des résultats

In [12]:
# Dashboard

def show_dossier(week,name):
    
    if name == "scen2006": 
        d = scen2006
        Volume = Volume_str_opti 
    else :
        d = pd.read_csv(name+"/"+name+"_hourly_generation.csv", index_col = 0, squeeze = True)
        Volume = pd.read_csv(name+"/"+name+"_Volume.csv", index_col = 0, squeeze = True)
        
    tec_aux = tec
    if not("river" in d.columns): tec_aux = list(set(tec_aux) - set(["river"]))
    if not("lake" in d.columns): tec_aux = list(set(tec_aux) - set(["lake"]))
    
    h  = 168*(week-1) + np.arange(1,169)
    ymin, ymax = -45,130
    h_ratio = np.ones(len(stor)+1)
    h_ratio[0] = 2 # 1er subplot 2 fois plus grand
    fig, axs = plt.subplots(len(stor)+1, figsize = (15,15),gridspec_kw={'height_ratios':h_ratio})
    
    # Stockage 
    sum_s = 0
    for i in stor: 
        sum_s += d["Storage " + i][h]
    for s in stor[::-1]:
        aux = "Storage " + s
        axs[0].fill_between(h, sum_s, color = col[s], label = aux)
        sum_s -= d[aux][h]

    # Production
    sum_g = 0
    for i in tec_aux: 
        sum_g += d[i][h]
    for gene in tec_aux[::-1]: 
        axs[0].fill_between(h,sum_g, color = col[gene], label = gene )
        sum_g -= d[gene][h]
    
    # Demande
    axs[0].plot(h, d["Electricity demand"][h], color = 'r' , label = "Demand",linewidth= 4)

    axs[0].vlines(x=np.linspace(h[0],h[-1],8),ymin = ymin, ymax = ymax,colors="black",linestyle='dotted',linewidth=2)
    axs[0].set_ylabel("Production (GWh)")
    axs[0].set_ylim(ymin,ymax)
    axs[0].legend(loc ="upper right")
    axs[0].set_title("Semaine "+ str(week))
        
    for s in range(0,len(stor)):
        axs[s+1].fill_between(h, d["Stored " + stor[s]][h], color = col[stor[s]])
        axs[s+1].hlines(y=Volume[s], xmin = h[0], xmax = h[-1], label = "limit", color = 'red')
        axs[s+1].set_title("Stored " + str(stor[s]))
        axs[s+1].vlines(x=np.linspace(h[0],h[-1],8),ymin = 0, ymax = Volume[s]+10,colors="black",linestyle='dotted',linewidth=2)
        
        
def affichage(simu):
    print(" --- Q ---")
    for i in simu.model.Q:
        print(i," : ",pyo.value(simu.model.Q[i]))   
    print(" --- S ---")   
    for i in simu.model.S:
        print(i," : ",pyo.value(simu.model.S[i]))
    print(" --- volume ---")
    for i in simu.model.volume:
        print(i," : ",pyo.value(simu.model.volume[i]))
    print(" --- cost --- ")
    print(simu.cost())

##### Composition du mix énergétique en fonction de la demande : ADEME ou NégaWatt

In [13]:
noms_simu = ["simu_opti_ademe", "simu_opti_negawatt","simu_opti_rte"]
# Dashboard
interactive_plot = interact_manual(show_dossier, week=widgets.IntText(value = 1), name = noms_simu)
interactive_plot

interactive(children=(IntText(value=1, description='week'), Dropdown(description='name', options=('simu_opti_a…

<function __main__.show_dossier(week, name)>

L'été on stocke l'énergie solaire en surplus la journée pour alimenter le réseau la nuit par PHS et batteries : court terme. On utilise également l'électricité générée par les barages.  
L'hiver, l'énergie solaire est moins présente, on génère de l'électricité avec l'éolien et on utilise massivement le stockage par méthanation : long terme. 